In [1]:
!pip install bitsandbytes -U
!pip install accelerate
!pip install punkt
!pip install datasets -U
!pip install evaluate -U
!pip install rouge_score -U

ERROR: Could not find a version that satisfies the requirement punkt (from versions: none)
ERROR: No matching distribution found for punkt


In [2]:
!pip install bert_score

Access tokens: https://huggingface.co/settings/tokens (if you don't have one, create in write mode)

In [16]:
#!huggingface-cli login
!huggingface-cli whoami

matteo1822


In [3]:
#Imports
import torch
import nltk
import json
from random import sample
import random
from datasets import load_dataset, DatasetDict
import evaluate
from transformers import AutoTokenizer, pipeline, BitsAndBytesConfig, AutoModelForCausalLM, TextGenerationPipeline

/home/matteo/AI_and_Robotics/NLP/HM3/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
SUBSET = 250 # number of sample to generate, over with compute the average metrics
SEED = 42
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 1
RESPONSE_TEMPLATE = "### Summary:"
PROMPT_TEMPLATE = "### Text:"

nltk.download('punkt')
random.seed(SEED)

[nltk_data] Downloading package punkt to /home/matteo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def postprocess_text(preds, labels, consider_just_n_sentence=1):
    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def generate_prompt_examples(texts): # TODO: implement this
    #print("TEXTS: ", texts)
    output_texts = []

    # Here you have to prompt the texts to be compatible with the format over which the model is trained

    # LOOK at the Trainer formatting...
    #I think
    for t in texts:
        new_text = f"{PROMPT_TEMPLATE} {texts}\n{RESPONSE_TEMPLATE}" #MAYBE FINE
        output_texts.append(new_text)
    """for i in range(len(example['text'])):
        text = f"{PROMPT_TEMPLATE} {example['text'][i]}\n{RESPONSE_TEMPLATE} {example['description'][i]}{tokenizer.eos_token}"
        output_texts.append(text)"""

    return output_texts

def call_model(examples, generator): # TODO: implement this
    """#text key replaces source key that fails
    print("TO DO 2")
    #print(examples)
    new_texts = generate_prompt_examples(examples["text"])

    # To implement this LOOK at this https://huggingface.co/docs/transformers/v4.41.3/en/main_classes/pipelines#transformers.TextGenerationPipeline

    generator = pipeline(model="openai-community/gpt2", text_inputs=[new_texts[0]])
    res = generator(new_texts[0], do_sample=False)
    print("TEXT: " + examples[0]["text"])
    print("NEW TEXT: " + new_texts[0])
    print("RES: " + res)

    outputs = []

    # Here you have to use the pipeline object to generate the summary of the sources that you can find in the examples list"""
    new_texts = generate_prompt_examples(examples["text"])
    outputs = []
    for t in new_texts:
      print("T: " + t)
      temp = generator(t)[0]["generated_text"]
      index = temp.find("Summary:")
      temp = temp[index+8:]
      print("OUTPUT: " + str(temp))
      outputs.append(temp)

    #print(examples)
    return {"text": examples["text"], "outputs":outputs ,"labels": examples["description"]}
    #return {"text": examples["text"], "outputs":outputs ,"labels": examples["target"]}


def map_samples(generator, samples):
    texts = []
    labels = []
    decoded_preds = []
    for item in samples.map(lambda e: call_model(e, generator), batched = True, batch_size = BATCH_SIZE, keep_in_memory=True, num_proc=1):
        decoded_preds.append(item["outputs"])
        labels.append(item["labels"])
        texts.append(item["text"])

    # Some simple post-processing
    decoded_preds_processed, labels_processed = postprocess_text(decoded_preds, labels)


    return texts, decoded_preds_processed, labels_processed

In [6]:
# DATASET
USE_SPLIT = "test"

dataset_ccnews = load_dataset("vblagoje/cc_news")
dataset_ccnews_filtered = dataset_ccnews.filter(lambda x: len(x["text"]) + len(x["description"]) < 2700 and len(x["description"]) > 0)
dataset_summary = DatasetDict()
dataset_summary["train"] = dataset_ccnews_filtered["train"].select(range(20000))
dataset_summary["test"] = dataset_ccnews_filtered["train"].select(range(20000, 25000)) # no test set defined on the original data, we create by ourselves

# generate indexes of samples to compute average evaluation metrics
evaluation_sample_idx = sample(list(range(len(dataset_summary[USE_SPLIT]))), SUBSET)
evaluation_samples = dataset_summary[USE_SPLIT].select(evaluation_sample_idx)

#cherry_picked_idx = [...] # modify this to select idx to qualitative evaluate the models
cherry_picked_idx = [1, 32, 56]
cherry_picked_samples = dataset_summary[USE_SPLIT].select(cherry_picked_idx)

In [7]:
# import eval metrics
rouge = evaluate.load('rouge') ## import the rouge scorer
bertscore = evaluate.load("bertscore")
bleu = evaluate.load("bleu")
meteor = evaluate.load('meteor')

[nltk_data] Downloading package wordnet to /home/matteo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/matteo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/matteo/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [19]:
# TEST STEPS x

MODEL_NAME = "matteo1822/minerva-1000"
REVISION_ID = "steps_1000"

#MODEL_NAME = "matteo1822/minerva-500"
#REVISION_ID = "steps_500"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, revision=REVISION_ID)
tokenizer.pad_token = tokenizer.unk_token
#tokenizer.padding_side = 'left'
tokenizer.padding_side = 'right'

# load the model in 4bit to have faster inference
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,
                                        quantization_config=bnb_config,
                                        revision=REVISION_ID,
                                        torch_dtype=torch.float16)

# TEXTGENERATIONPIPELINE
generator = TextGenerationPipeline(
                model=model, tokenizer=tokenizer, max_new_tokens=500
            )

#generator = pipeline("text-generation",model=model, tokenizer=tokenizer, max_new_tokens=500)

generator.device = model.device

print(f"DEVICE:\n\tSelected Device: {DEVICE}\n\tmodel Device: {model.device}\n\tpipeline Device: {generator.device}")

ImportError: 
LlamaConverter requires the protobuf library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/protocolbuffers/protobuf/tree/master/python#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [ ]:
# generate output over subsamples of the test set
texts, decoded_preds_processed, labels_processed = map_samples(generator, evaluation_samples)

In [ ]:
# Compute ROUGE METRICS over the entire generated samples
result = rouge.compute(predictions=decoded_preds_processed, references=labels_processed, use_stemmer=True)
rouge_res = {k: round(v * 100, 4) for k, v in result.items()}
bertscore_res = bertscore.compute(predictions=decoded_preds_processed, references=labels_processed, lang="en")
bleu_res = bleu.compute(predictions=decoded_preds_processed, references=labels_processed)
meteor_res = meteor.compute(predictions=decoded_preds_processed, references=labels_processed)

# ADD other METRICS: leave the students the way of find and compare different summarization metrics
# ... # add to the result dictionary

print("="*30)
print(f"SPLIT: {USE_SPLIT}")
print(rouge_res)
print(bertscore_res)
print(bleu_res)
print(meteor_res)
print("="*30)

# save average metrics
with open(f"average_metric_x.jsonl", "w") as f:
    f.writelines(json.dumps(result) + "\n")

# generate output over the cherry picked samples
texts, decoded_preds_processed, labels_processed = map_samples(generator, cherry_picked_samples)
for i in range(3):
  print("TEXTS: " + texts[i])
  print("DECODED: " + decoded_preds_processed[i])
  print("LABELS: " + labels_processed[i])

In [ ]:
# save selected samples
with open("Checkpoint_x_selected_samples.jsonl", "w") as f:
    for text, dec, lab in zip(texts, decoded_preds_processed, labels_processed):

        rouge_res = rouge.compute(predictions=[dec], references=[lab], use_stemmer=True)
        bertscore_res = bertscore.compute(predictions=[dec], references=[lab], lang="en")
        bleu_res = bleu.compute(predictions=[dec], references=[lab])
        meteor_res = meteor.compute(predictions=[dec], references=[lab])

        # modify this to add more metrics
        f.writelines(json.dumps({"text": text, "gold": lab, "predicted": dec, "rouge1": rouge_res["rouge1"], "rouge2": rouge_res["rouge2"], "rougeL": rouge_res["rougeL"], "rougeLsum": rouge_res["rougeLsum"], "bertscore_precision": bertscore_res["precision"], "bertscore_recall": bertscore_res["recall"], "bertscore_F1": bertscore_res["f1"], "bleu": bleu_res["bleu"], "meteor": meteor_res["meteor"]}) + "\n")
        #f.writelines(json.dumps({"text": text, "gold": lab, "predicted": dec, "rouge1": result["rouge1"], ...}) + "\n") # TODO: add metrics there

In [ ]:
# TEST STEPS y ????

...

### What you have to do

1. Compute the average of the choosen metrics over a reasonable amount of istances of the test set for two selected checkpoints.

2. Report and save, for every checkpoint, 3 different samples. For each sample you must provide: the input text, the gold summary, the summary generated with that checkpoint, and the associated metrics (at least the Rouge ones).

3. For the 3 chosen examples, you have to highlight: hallucinations, quality of the generated text, factual references on the input, repetitions and never ending generation.

4. Additionally, for every checkpoint, provide your own insights about their generated summaries with respect to the input examples.

5. (EXTRA) You have to choose two additional metrics other than Rouge ones to evaluate the models' outputs, providing insights about the relationships of the resulting scores with respect to the theory behind these metrics.
